In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [3]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [5]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.306843  [    0/60000]
loss: 2.299320  [ 6400/60000]
loss: 2.276786  [12800/60000]
loss: 2.267307  [19200/60000]
loss: 2.251402  [25600/60000]
loss: 2.217239  [32000/60000]
loss: 2.230764  [38400/60000]
loss: 2.184378  [44800/60000]
loss: 2.178214  [51200/60000]
loss: 2.153976  [57600/60000]
Test Error: 
 Accuracy: 41.7%, Avg loss: 2.149521 

Epoch 2
-------------------------------
loss: 2.156437  [    0/60000]
loss: 2.153442  [ 6400/60000]
loss: 2.092695  [12800/60000]
loss: 2.109426  [19200/60000]
loss: 2.060611  [25600/60000]
loss: 1.994474  [32000/60000]
loss: 2.028152  [38400/60000]
loss: 1.936394  [44800/60000]
loss: 1.941172  [51200/60000]
loss: 1.875435  [57600/60000]
Test Error: 
 Accuracy: 60.2%, Avg loss: 1.874850 

Epoch 3
-------------------------------
loss: 1.907207  [    0/60000]
loss: 1.881860  [ 6400/60000]
loss: 1.760405  [12800/60000]
loss: 1.798911  [19200/60000]
loss: 1.698941  [25600/60000]
loss: 1.644934  [32000/600

KeyboardInterrupt: 